In [25]:
#Create labels for classifying stock price changes

def bucket(perc_change):
    if perc_change<-2:
        return -2
    elif perc_change<0:
        return -1
    elif perc_change<2:
        return 1
    else:
        return 2

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

file_path = 'final_data.csv'
data = pd.read_csv(file_path).dropna()
data['Month'] = data['Month'].apply(lambda x : int(x))
data['Month'] = data['Month'].astype('category')
data['perc_change_bucket'] = data['perc_change_next_prev'].apply(bucket)

data_train, data_test = train_test_split(data, test_size=0.2, shuffle=True, stratify=data['Ticker'])

In [27]:
data[['perc_change_bucket', 'perc_change_next_prev']].sample(10)

,perc_change_bucket,perc_change_next_prev
1889,-2,-3.23
964,2,4.40
19,2,2.26
2064,1,0.26
1391,2,3.54
1033,-2,-5.29
1305,-2,-2.49
923,-2,-4.45
553,2,7.00
2121,-2,-6.99


In [28]:
relevant_cols = ['average_volume_50_days', 'quant_score_NER',
       'financial_performance_score', 'market_position_score',
       'strategic_direction_score', 'operational_aspects_score',
       'financial_indicators_score', 'risks_challenges_score',
       'economic_factors_score', '% Change Revenue', '% Change EPS GAAP',
       '% Change EPS Normalized',
       'perc_change_prev_7', 'perc_change_7_15']

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

scaler = StandardScaler()
scaler.fit(data_train[relevant_cols])
data_train[relevant_cols] = scaler.transform(data_train[relevant_cols])


months_onehot_train = pd.get_dummies(data=data_train['Month'], dtype=int)
months_onehot_train.columns = months_onehot_train.columns.astype(str)
data_train_final = pd.concat([data_train[relevant_cols], months_onehot_train], axis=1)


In [30]:
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression()
logreg.fit(data_train_final, data_train['perc_change_bucket'])

LogisticRegression()

In [31]:
data_test[relevant_cols] = scaler.transform(data_test[relevant_cols])

months_onehot_test = pd.get_dummies(data=data_test['Month'], dtype=int)
months_onehot_test.columns = months_onehot_test.columns.astype(str)
data_test_final = pd.concat([data_test[relevant_cols], months_onehot_test], axis=1)


y_predict = logreg.predict(data_test_final)

In [32]:
from sklearn.metrics import confusion_matrix

confusion_matrix(data_test['perc_change_bucket'], y_predict)

array([[ 58,   3,   4,  69],
       [ 31,   1,   2,  47],
       [ 24,   1,   3,  47],
       [ 40,   3,   4, 114]])